## <font color = "deepskyblue"> 1) Install necessary modules and packages and libraries </font>


In [ ]:
#installing necessary dependencies and packages 
!pip install pyserial
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!pip install gspread

## <font color = "deepskyblue"> 2) Once you have your Google Spreadsheet and API credentials ready, replace the fileds based on directories, port numbers, etc on your computer and run the code below</font>

In [ ]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# the data coming from serial monitor is separated by "," and stored in one single string
# Define the column headers based on your desired CSV structure

column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Here we setup the Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file(the file you downloaded making API on google console service)
CREDS_FILE = r""

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Replace with the ID of your Google Sheets document
# open your spreadsheet on your Google drive, in the address bar, copy the ID which is between "/d/" and "/edit"
SPREADSHEET_ID = ""

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def read_from_port(ser, worksheet_name):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        # to sync all the FED units with each other, I tend not to rely on FED clock,
        # Ignore the first field (timestamp from FED device) and use the computer's timestamp
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        # Assuming the data matches the order of the remaining column_headers
        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Append the row to Google Sheet
            sheet.append_row([timestamp] + data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
ports = ["COM5","COM12","COM16","COM19"]  # Replace with your COM ports, on Mac systems the port number is different and longer
baud_rate = 115200

# Start reading from each port in a separate thread
for port in ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name)).start()


In [1]:
import serial
import threading
import datetime  
import gspread
from google.oauth2.service_account import Credentials

# the data coming from serial monitor is separated by "," and stored in one single string
# Define the column headers based on your desired CSV structure

column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Here we setup the Google Sheets
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Path to your downloaded JSON file(the file you downloaded making API on google console service)
CREDS_FILE = r"C:\Users\hta031\Github\rtfed-432707-fe18d041b295.json"

creds = Credentials.from_service_account_file(CREDS_FILE, scopes=SCOPE)
client = gspread.authorize(creds)

# Replace with the ID of your Google Sheets document
# open your spreadsheet on your Google drive, in the address bar, copy the ID which is between "/d/" and "/edit"
SPREADSHEET_ID = "1oybqWp_7b9_oiR-a1Xy0YLw8LwvGfqtmSz2lYfEzrBk"

def get_or_create_worksheet(spreadsheet, title):
    try:
        # Try to open the worksheet by title
        sheet = spreadsheet.worksheet(title)
        print(f"Worksheet '{title}' found.")
    except gspread.exceptions.WorksheetNotFound:
        # If the worksheet is not found, create it
        print(f"Worksheet '{title}' not found. Creating a new one.")
        sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
        sheet.append_row(column_headers)
    return sheet

def read_from_port(ser, worksheet_name):
    spreadsheet = client.open_by_key(SPREADSHEET_ID)
    sheet = get_or_create_worksheet(spreadsheet, worksheet_name)
    
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")  # Split the data string into a list
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp with milliseconds
        # to sync all the FED units with each other, I tend not to rely on FED clock,
        # Ignore the first field (timestamp from FED device) and use the computer's timestamp
        data_list = data_list[1:]  # Skip the FED device timestamp
        
        print(f"Data from {ser.port}: {data}")

        # Assuming the data matches the order of the remaining column_headers
        if len(data_list) == len(column_headers) - 1:  # -1 because timestamp is added
            # Append the row to Google Sheet
            sheet.append_row([timestamp] + data_list)
        else:
            print(f"Warning: Data length {len(data_list)} does not match header length {len(column_headers) - 1}")

# Define your ports and baud rate
ports = ["COM5","COM12","COM16","COM19"]  # Replace with your COM ports, on Mac systems the port number is different and longer
baud_rate = 115200

# Start reading from each port in a separate thread
for port in ports:
    worksheet_name = f"Port_{port}"  # Create a unique worksheet name for each port
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, worksheet_name)).start()


Worksheet 'Port_COM12' not found. Creating a new one.
Worksheet 'Port_COM16' not found. Creating a new one.
Worksheet 'Port_COM19' not found. Creating a new one.
Worksheet 'Port_COM5' not found. Creating a new one.
Data from COM5: 8/29/2024 10:52:22,25.47,56.35,1.16.3,FR1,4,3.63,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.00
Data from COM5: 8/29/2024 10:52:28,25.49,56.37,1.16.3,FR1,4,3.63,NaN,1,Right,Left,0,2,0,0,NaN,NaN,0.01
Data from COM5: 8/29/2024 10:52:30,25.47,56.38,1.16.3,FR1,4,3.63,NaN,1,Right,Left,0,3,0,0,NaN,NaN,0.05
Data from COM5: 8/29/2024 10:52:31,25.50,56.40,1.16.3,FR1,4,3.63,NaN,1,Right,Left,0,4,0,0,NaN,NaN,0.00
Data from COM5: 8/29/2024 10:52:32,25.49,56.37,1.16.3,FR1,4,3.64,NaN,1,Right,Left,0,5,0,0,NaN,NaN,0.00
Data from COM5: 8/29/2024 10:52:33,25.50,56.41,1.16.3,FR1,4,3.63,NaN,1,Right,Left,0,6,0,0,NaN,NaN,0.07
Data from COM19: 8/29/2024 10:57:02,25.31,55.11,1.16.3,FR1,11,3.76,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.00
Data from COM19: 8/29/2024 10:57:05,25.30,55.11,1.16.3,FR1,11,

Exception in thread Thread-7 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_96852\906116238.py", line 48, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Data from COM16: 


Exception in thread Thread-6 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_96852\906116238.py", line 48, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Data from COM12: 


Exception in thread Thread-8 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_96852\906116238.py", line 48, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Data from COM19: 


Exception in thread Thread-5 (read_from_port):
Traceback (most recent call last):
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hta031\AppData\Local\Temp\ipykernel_96852\906116238.py", line 48, in read_from_port
  File "C:\Users\hta031\AppData\Local\miniconda3\envs\RTFED\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))


Data from COM5: 
